In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Version info: sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)


In [2]:
# Read the dataset files list
file_list = pd.read_csv('../Dataset/file_list.csv')
file_list.head()

,File,Category,Attack,Class
0,ARP_Spoofing_train.pcap.csv,SPOOFING,ARP Spoofing,1
1,Benign_train.pcap.csv,BENIGN,Benign,0
2,MQTT-DDoS-Connect_Flood_train.pcap.csv,MQTT,DDoS Connect Flood,1
3,MQTT-DDoS-Publish_Flood_train.pcap.csv,MQTT,DDoS Publish Flood,1
4,MQTT-DoS-Connect_Flood_train.pcap.csv,MQTT,DoS Connect Flood,1


In [3]:
# Import the dataset
data_dir = "../Dataset"
first_file_path = os.path.join(data_dir, file_list.iloc[0]['File'])
column_names = pd.read_csv(first_file_path, nrows=0).columns.tolist()

datasets_list = [] 
for _, row in file_list.iterrows():
    file_path = os.path.join(data_dir, row['File'])
    if os.path.exists(file_path):
        data = pd.read_csv(file_path, header=None, skiprows=1, names=column_names)
        data['Class'] = row['Class']
        datasets_list.append(data)
    else:
        print(f"File not found: {file_path}")

# Combine all datasets into a single DataFrame
dataset = pd.concat(datasets_list, ignore_index=True)
dataset

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Class
0,866.600000,10.4,64.0,45722.390222,45722.390222,0.0,0.0,0.0,0.0,0.3,...,150.681076,431.800000,1.694026e+08,5.500000,16.963645,213.095221,66236.076476,0.900000,38.500000,1
1,3934.300000,12.6,131.2,35708.799475,35708.799475,0.0,0.0,0.0,0.0,0.3,...,500.702909,406.300000,1.694026e+08,13.500000,30.885371,708.919620,251721.126817,1.000000,244.600000,1
2,5592.800000,12.6,97.6,66.403506,66.403506,0.0,0.0,0.0,0.0,0.3,...,205.552035,386.600000,1.361110e-02,5.500000,21.787095,290.694475,84028.647525,0.900000,38.500000,1
3,9303.600000,14.8,80.8,51.201280,51.201280,0.0,0.0,0.0,0.0,0.1,...,421.683660,300.200000,1.694026e+08,13.500000,26.954506,597.046005,178453.001691,1.000000,244.600000,1
4,8592.400000,12.6,98.6,42.706455,42.706455,0.0,0.0,0.0,0.0,0.2,...,265.394239,209.200000,1.393099e-02,5.500000,24.255148,375.324132,80115.110731,0.900000,38.500000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7160826,13234.910000,17.0,64.0,29460.045053,29460.045053,0.0,0.0,0.0,0.0,0.0,...,5.968139,51.740000,8.467524e+07,9.500000,10.164175,8.453393,279.410488,0.130000,141.550000,1
7160827,18136.530000,17.0,64.0,29318.535060,29318.535060,0.0,0.0,0.0,0.0,0.0,...,2.126578,50.620000,8.467524e+07,9.500000,10.060924,3.012129,35.475423,0.130000,141.550000,1
7160828,23038.040000,17.0,64.0,29363.556139,29363.556139,0.0,0.0,0.0,0.0,0.0,...,2.321811,50.510000,8.467524e+07,9.500000,10.075262,3.287396,34.963050,0.160000,141.550000,1
7160829,58526.160000,17.0,64.0,26168.955034,26168.955034,0.0,0.0,0.0,0.0,0.0,...,147.975762,170.680000,8.467524e+07,9.500000,18.631072,205.641671,70932.249555,0.350000,141.550000,1


In [5]:
# Select 'Category' to perform multiclass classification
dataset.rename(columns={'Class': 'Label'}, inplace=True)

selected_data=dataset
selected_data.head()

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Label
0,866.6,10.4,64.0,45722.390222,45722.390222,0.0,0.0,0.0,0.0,0.3,...,150.681076,431.8,1.694026e+08,5.5,16.963645,213.095221,66236.076476,0.9,38.5,1
1,3934.3,12.6,131.2,35708.799475,35708.799475,0.0,0.0,0.0,0.0,0.3,...,500.702909,406.3,1.694026e+08,13.5,30.885371,708.919620,251721.126817,1.0,244.6,1
2,5592.8,12.6,97.6,66.403506,66.403506,0.0,0.0,0.0,0.0,0.3,...,205.552035,386.6,1.361110e-02,5.5,21.787095,290.694475,84028.647525,0.9,38.5,1
3,9303.6,14.8,80.8,51.201280,51.201280,0.0,0.0,0.0,0.0,0.1,...,421.683660,300.2,1.694026e+08,13.5,26.954506,597.046005,178453.001691,1.0,244.6,1
4,8592.4,12.6,98.6,42.706455,42.706455,0.0,0.0,0.0,0.0,0.2,...,265.394239,209.2,1.393099e-02,5.5,24.255148,375.324132,80115.110731,0.9,38.5,1


In [6]:
selected_data['Label'].value_counts()

Label
1    6968099
0     192732
Name: count, dtype: int64

In [7]:
X = selected_data.iloc[:, :-1]
y = selected_data.iloc[:, -1]
class_counts = Counter(y)
print("Class distribution:", class_counts)

undersampler = RandomUnderSampler(random_state=42)
X_balanced, y_balanced = undersampler.fit_resample(X, y)

processed_data = pd.DataFrame(X_balanced, columns=X.columns)
processed_data['Label'] = y_balanced
processed_data.head()

Class distribution: Counter({1: 6968099, 0: 192732})


,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Label
16047,134.3,7.1,64.0,40415.288318,40415.288318,0.0,0.0,0.0,0.0,0.5,...,48.225852,95.3,1.694641e+08,5.5,15.312231,68.201654,2644.242675,0.9,38.5,0
16048,137.0,4.7,49.9,58749.191833,58749.191833,0.0,0.0,0.0,0.0,0.2,...,59.753361,125.6,1.694641e+08,13.5,13.468364,84.997147,5328.483794,1.0,244.6,0
16049,283.9,6.0,64.0,3.497439,3.497439,0.0,0.0,0.0,0.0,0.5,...,10.223690,80.6,8.070469e-05,5.5,12.229315,14.458481,124.725884,0.9,38.5,0
16050,297.3,6.0,64.0,7297.703020,7297.703020,0.0,0.0,0.0,0.0,0.5,...,15.625023,78.5,1.694641e+08,13.5,12.552920,21.904691,240.536207,1.0,244.6,0
16051,572.3,7.1,57.7,2.549120,2.549120,0.0,0.0,0.0,0.0,0.4,...,69.738867,139.0,9.993820e-02,5.5,13.928729,98.625651,11737.768847,0.9,38.5,0


In [8]:
processed_data['Label'].value_counts()

Label
0    192732
1    192732
Name: count, dtype: int64

In [10]:
# Normalize the data
scaler = MinMaxScaler()
normalize_data = processed_data.columns[:-1]
processed_data[normalize_data] = scaler.fit_transform(processed_data[normalize_data])
processed_data.head()

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Label
16047,0.000014,0.417647,0.250980,0.019272,0.019272,0.0,0.0,0.0,0.0,0.5,...,0.066873,0.036209,9.999685e-01,0.321429,0.134034,0.066849,0.005081,0.9,38.5,0
16048,0.000014,0.276471,0.195686,0.028014,0.028014,0.0,0.0,0.0,0.0,0.2,...,0.082858,0.056793,9.999685e-01,0.892857,0.093829,0.083312,0.010238,1.0,244.6,0
16049,0.000029,0.352941,0.250980,0.000002,0.000002,0.0,0.0,0.0,0.0,0.5,...,0.014177,0.026223,7.565624e-09,0.321429,0.066813,0.014172,0.000240,0.9,38.5,0
16050,0.000030,0.352941,0.250980,0.003480,0.003480,0.0,0.0,0.0,0.0,0.5,...,0.021667,0.024796,9.999685e-01,0.892857,0.073869,0.021470,0.000462,1.0,244.6,0
16051,0.000058,0.417647,0.226275,0.000001,0.000001,0.0,0.0,0.0,0.0,0.4,...,0.096705,0.065897,8.154860e-09,0.321429,0.103867,0.096670,0.022554,0.9,38.5,0


In [15]:
# Assign clients ID
processed_data["Client_ID"] = np.random.randint(1, 6, size=len(processed_data))
processed_data["Client_ID"].value_counts()

Client_ID
1    77526
4    77183
2    77096
3    76905
5    76754
Name: count, dtype: int64

In [16]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 385464 entries, 16047 to 4495770
Data columns (total 47 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Header_Length    385464 non-null  float64
 1   Protocol Type    385464 non-null  float64
 2   Duration         385464 non-null  float64
 3   Rate             385464 non-null  float64
 4   Srate            385464 non-null  float64
 5   Drate            385464 non-null  float64
 6   fin_flag_number  385464 non-null  float64
 7   syn_flag_number  385464 non-null  float64
 8   rst_flag_number  385464 non-null  float64
 9   psh_flag_number  385464 non-null  float64
 10  ack_flag_number  385464 non-null  float64
 11  ece_flag_number  385464 non-null  float64
 12  cwr_flag_number  385464 non-null  float64
 13  ack_count        385464 non-null  float64
 14  syn_count        385464 non-null  float64
 15  fin_count        385464 non-null  float64
 16  rst_count        385464 non-null  floa

In [17]:
# Shuffle the dataset
processed_data = processed_data.sample(frac=1).reset_index(drop=True)
processed_data.head()

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,Label,Client_ID
0,0.012120,0.352941,0.250980,3.666254e-07,3.666254e-07,0.0,0.0,0.0,0.0,0.0,...,0.016304,9.999843e-01,0.892857,0.079880,0.023133,0.000535,1.0,244.6,0,3
1,0.311389,0.352941,0.215686,1.741823e-03,1.741823e-03,0.0,0.0,0.0,0.0,0.0,...,1.000000,7.565949e-09,0.321429,1.000000,0.000000,0.000000,0.0,38.5,0,3
2,0.000141,0.352941,0.250980,1.192039e-06,1.192039e-06,0.0,0.0,0.0,0.0,0.5,...,0.026155,9.999820e-01,0.892857,0.080789,0.052023,0.002719,1.0,244.6,0,4
3,0.037971,0.417647,0.226275,9.358538e-07,9.358538e-07,0.0,0.0,0.0,0.0,0.5,...,0.034511,9.999847e-01,0.892857,0.093992,0.058092,0.003398,1.0,244.6,0,4
4,0.002925,0.352941,0.316471,8.763341e-07,8.763341e-07,0.0,0.0,0.0,0.0,0.5,...,0.026223,8.155264e-09,0.321429,0.069141,0.016860,0.000346,0.9,38.5,0,4


In [18]:
processed_data['Label'].value_counts()

Label
0    192732
1    192732
Name: count, dtype: int64

In [19]:
processed_data['Client_ID'].value_counts()

Client_ID
1    77526
4    77183
2    77096
3    76905
5    76754
Name: count, dtype: int64

In [20]:
processed_data.to_csv("binary.csv", index=False, header=True)